- NVIDIA GeForce GTX 1060 6GB
- Pytorch 1.4.0
- model EfficientNet-B3
- image size 128x128
- batch size 64
- 5-folds
- 10 epochs
- no augmentation
- optimizer over 9000
- one cycle learning rate

In [25]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import gc
import cv2
import json
#import math
import numpy as np
import pandas as pd
#import six
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
#from typing import List
import torch
#from torch.nn import init
#from torch.nn.parameter import Parameter
#import torch.nn.functional as F
#from torch.nn import Sequential
#import pretrainedmodels
import warnings

from crop_resize import Resize, read_feathers

In [2]:
image_size = 128
#debug = False
#submission = False
batch_size = 64
#device = 'cuda:0'
#out = '.'
#arch = 'pretrained'
#model_name = 'se_resnext50_32x4d'
model_name = 'efficientnet-b3'
#indices=[0, 1, 2, 3]
train_size = 0.8
random_state = 2020
n_epochs = 10

in_dir = Path('../input/bengaliai-cv19')
feather_dir = Path('../input/bengaliai-cv19-feather')
out_dir = Path('./20200307_')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

<a id="images"></a>
# Images

In [4]:
filenames = [feather_dir/f'train_image_data_1x{image_size}x{image_size}_{i}.feather' for i in range(4)]
images = read_feathers(filenames, image_size)
print(images.shape)

(200840, 128, 128)


<a id="labels"></a>
# labels

In [5]:
train = pd.read_csv(in_dir/'train.csv')
labels = train[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values
print(labels.shape)

(200840, 3)


<a id="dataset"></a>
# Dataset

In [6]:
from torch.utils.data import Dataset

class GraphemeDataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.train = labels is not None
    
    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        if self.train:
            label = self.labels[idx]
            return image, label[0], label[1], label[2]
        else:
            return image

In [7]:
#images = images[:10]
#labels = labels[:10]

In [7]:
from sklearn.model_selection import train_test_split

train_images, valid_images, train_labels, valid_labels = train_test_split(images, labels, train_size=train_size,
                                                                          random_state=random_state, shuffle=True)

print('images split')
print(train_images.shape)
print(valid_images.shape)

print('labels split')
print(train_labels.shape)
print(valid_labels.shape)

images split
(160672, 128, 128)
(40168, 128, 128)
labels split
(160672, 3)
(40168, 3)


In [8]:
train_dataset = GraphemeDataset(train_images, train_labels)
valid_dataset = GraphemeDataset(valid_images, valid_labels)

In [9]:
del train_images, train_labels, valid_images, valid_labels
torch.cuda.empty_cache()
gc.collect()

40

<a id="dataloader"></a>
# DataLoader

In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

print('train loader length', len(train_loader))
print('valid loader length', len(valid_loader))

train loader length 2511
valid loader length 628


<a id="model"></a> 
# Model

In [11]:
from my_efficientnet_pytorch import EfficientNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = EfficientNet.from_pretrained(model_name, in_channels=1).to(device)

cuda:0
missing keys :  ['_fc1.weight', '_fc1.bias', '_fc2.weight', '_fc2.bias', '_fc3.weight', '_fc3.bias']
Loaded pretrained weights for efficientnet-b3


<a id="optimizer"></a> 
# Optimizer

In [12]:
from optimizer.ralamb import Ralamb
from optimizer.lookahead import Lookahead

#def LookaheadAdam(params, alpha=0.5, k=6, *args, **kwargs):
#     adam = Adam(params, *args, **kwargs)
#     return Lookahead(adam, alpha, k)

def Over9000(params, alpha=0.5, k=6, *args, **kwargs):
     ralamb = Ralamb(params, *args, **kwargs)
     return Lookahead(ralamb, alpha, k)

#RangerLars = Over9000

In [13]:
optimizer =Over9000(model.parameters(), lr=2e-3, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2, total_steps=None, epochs=n_epochs, 
                                                steps_per_epoch=len(train_loader), pct_start=0.0, anneal_strategy='cos', 
                                                cycle_momentum=True, base_momentum=0.85, max_momentum=0.95,  div_factor=100.0)

<a id="loss"></a> 
# loss

In [14]:
from torch import nn

criterion = nn.CrossEntropyLoss()

<a id="metrics"></a> 
# Metrics

In [15]:
from metrics import macro_recall_multi

<a id="train"></a> 
# Train

In [16]:
history = pd.DataFrame()
best_valid_recall = 0.0

In [17]:
for epoch in range(n_epochs):
    
    # --- training start ---
    torch.cuda.empty_cache()
    gc.collect()
    
    running_loss, running_acc, running_recall = 0.0, 0.0, 0.0
    train_loss, train_acc, train_recall = 0.0, 0.0, 0.0
    model.train()
    
    # training loop
    for idx, (inputs, labels1, labels2, labels3) in tqdm(enumerate(train_loader), total=len(train_loader)):
        
        # to GPU
        inputs, labels1, labels2, labels3 = inputs.to(device), labels1.to(device), labels2.to(device), labels3.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward
        outputs1, outputs2, outputs3 = model(inputs.unsqueeze(1).float())
        loss1 = 0.7 * criterion(outputs1, labels1)
        loss2 = 0.1 * criterion(outputs2, labels2)
        loss3 = 0.2 * criterion(outputs3, labels3)
        running_loss += loss1.item() + loss2.item() + loss3.item()
        running_acc += (outputs1.argmax(1)==labels1).float().mean()
        running_acc += (outputs2.argmax(1)==labels2).float().mean()
        running_acc += (outputs3.argmax(1)==labels3).float().mean()
        running_recall += macro_recall_multi(outputs1, labels1, outputs2, labels2, outputs3, labels3)
        
        # backward
        (loss1 + loss2 + loss3).backward()
        
        # optimize
        optimizer.step()
        scheduler.step()
        
    train_loss = running_loss / len(train_loader)
    train_acc = running_acc / (len(train_loader) * 3)
    train_recall = running_recall / len(train_loader)
    
    print('train epoch  : {}'.format(epoch))
    print('      loss   : {:.4f}'.format(train_loss))
    print('      acc    : {:.4f}'.format(train_acc))
    print('      recall : {:.4f}'.format(train_recall))

    history.loc[epoch, 'train_loss'] = train_loss
    history.loc[epoch, 'train_acc'] = train_acc.cpu().numpy()
    history.loc[epoch, 'train_recall'] = train_recall
    
    # --- validation start ---
    torch.cuda.empty_cache()
    gc.collect()
    
    running_loss, running_acc, running_recall = 0.0, 0.0, 0.0
    valid_loss, valid_acc, valid_recall = 0.0, 0.0, 0.0
    model.eval()
    
    with torch.no_grad():
        
        for idx, (inputs, labels1, labels2, labels3) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
            
            # to GPU
            inputs, labels1, labels2, labels3 = inputs.to(device), labels1.to(device), labels2.to(device), labels3.to(device)
            
            # forward
            outputs1, outputs2, outputs3 = model(inputs.unsqueeze(1).float())
            loss1 = 2.0 * criterion(outputs1, labels1)
            loss2 = 1.0 * criterion(outputs2, labels2)
            loss3 = 1.0 * criterion(outputs3, labels3)
            running_loss += loss1.item() + loss2.item() + loss3.item()
            running_acc += (outputs1.argmax(1)==labels1).float().mean()
            running_acc += (outputs2.argmax(1)==labels2).float().mean()
            running_acc += (outputs3.argmax(1)==labels3).float().mean()
            running_recall += macro_recall_multi(outputs1, labels1, outputs2, labels2, outputs3, labels3)
            
    valid_loss = running_loss / len(valid_loader)
    valid_acc = running_acc / (len(valid_loader) * 3)
    valid_recall = running_recall / len(valid_loader)
    
    print('valid epoch  : {}'.format(epoch))
    print('      loss   : {:.4f}'.format(valid_loss))
    print('      acc    : {:.4f}'.format(valid_acc))
    print('      recall : {:.4f}'.format(valid_recall))
    
    history.loc[epoch, 'valid_loss'] = valid_loss
    history.loc[epoch, 'valid_acc'] = valid_acc.cpu().numpy()
    history.loc[epoch, 'valid_recall'] = valid_recall
    
    if valid_recall > best_valid_recall:
        print(f'validation recall has increased from: {best_valid_recall:.4f} to: {valid_recall:.4f}. Saving checkpoint')
        torch.save(model.state_dict(), out_dir/f'efficientnet-b0_{epoch}.pth')
        best_valid_recall = valid_recall

C:\Users\naoya\Anaconda3\envs\fastai-v1\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train epoch  : 0
      loss   : 0.7165
      acc    : 0.8461
      recall : 0.7504



valid epoch  : 0
      loss   : 1.4806
      acc    : 0.9107
      recall : 0.8378
validation recall has increased from: 0.0000 to: 0.8378. Saving checkpoint



train epoch  : 1
      loss   : 0.2952
      acc    : 0.9355
      recall : 0.8799



valid epoch  : 1
      loss   : 0.8157
      acc    : 0.9504
      recall : 0.9041
validation recall has increased from: 0.8378 to: 0.9041. Saving checkpoint


...


valid epoch  : 7
      loss   : 0.5136
      acc    : 0.9762
      recall : 0.9507
validation recall has increased from: 0.9481 to: 0.9507. Saving checkpoint



train epoch  : 8
      loss   : 0.0129
      acc    : 0.9952
      recall : 0.9928



valid epoch  : 8
      loss   : 0.5260
      acc    : 0.9770
      recall : 0.9528
validation recall has increased from: 0.9507 to: 0.9528. Saving checkpoint



train epoch  : 9
      loss   : 0.0092
      acc    : 0.9965
      recall : 0.9953



valid epoch  : 9
      loss   : 0.5299
      acc    : 0.9771
      recall : 0.9529
validation recall has increased from: 0.9528 to: 0.9529. Saving checkpoint


In [19]:
history.to_csv(out_dir/'history.csv')
history

,train_loss,train_acc,train_recall,valid_loss,valid_acc,valid_recall
0,0.716531,0.846125,0.750375,1.480586,0.910677,0.837821
1,0.295248,0.935546,0.879895,0.815661,0.950365,0.904117
2,0.215565,0.952308,0.909651,0.941641,0.943445,0.896709
3,0.158839,0.963496,0.930600,0.618874,0.963859,0.928866
4,0.113270,0.972967,0.948769,0.543620,0.969775,0.939794
5,0.074646,0.980994,0.964523,0.491962,0.973413,0.947245
6,0.043740,0.987437,0.977232,0.504865,0.974458,0.948061
7,0.023931,0.992226,0.987014,0.513597,0.976183,0.950704
8,0.012878,0.995238,0.992752,0.525973,0.976962,0.952753
9,0.009215,0.996466,0.995267,0.529881,0.977087,0.952942


<a id="inference"></a> 
# Inference

In [29]:
from crop_resize import read_parquets

height = 137
width = 236
image_size = 128

filenames = [in_dir/f'test_image_data_{i}.parquet' for i in range(4)]
filenames

[WindowsPath('../input/bengaliai-cv19/test_image_data_0.parquet'),
 WindowsPath('../input/bengaliai-cv19/test_image_data_1.parquet'),
 WindowsPath('../input/bengaliai-cv19/test_image_data_2.parquet'),
 WindowsPath('../input/bengaliai-cv19/test_image_data_3.parquet')]

In [32]:
images = read_parquets(filenames, width, height, image_size)
images.shape

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1500.47it/s]


(12, 128, 128)

In [34]:
test_dataset = GraphemeDataset(images)
print(len(test_dataset))

12


In [36]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(len(test_loader))

12


In [51]:
row_id = []
target = []

for idx, inputs in tqdm(enumerate(test_loader), total=len(test_loader)):
    outputs1, outputs2, outputs3 = model(inputs.unsqueeze(1).float().cuda())
    p1 = outputs1.argmax(-1).view(-1).cpu()
    p2 = outputs2.argmax(-1).view(-1).cpu()
    p3 = outputs3.argmax(-1).view(-1).cpu()
    row_id += [f'Test_{idx}_grapheme_root', f'Test_{idx}_vowel_diacritic', f'Test_{idx}_consonant_diacritic']
    target += [p1.item(),p2.item(),p3.item()]

sub_df = pd.DataFrame({'row_id': row_id, 'target': target})
sub_df.to_csv(out_dir/'submission.csv', index=False)
sub_df.head(20)

,row_id,target
0,Test_0_grapheme_root,3
1,Test_0_vowel_diacritic,0
2,Test_0_consonant_diacritic,0
3,Test_1_grapheme_root,93
4,Test_1_vowel_diacritic,2
5,Test_1_consonant_diacritic,0
6,Test_2_grapheme_root,19
7,Test_2_vowel_diacritic,0
8,Test_2_consonant_diacritic,0
9,Test_3_grapheme_root,115


In [ ]:
for epoch in range(n_epochs):
    
    # --- training start ---
    torch.cuda.empty_cache()
    gc.collect()
    
    running_loss, running_acc, running_recall = 0.0, 0.0, 0.0
    train_loss, train_acc, train_recall = 0.0, 0.0, 0.0
    model.train()
    
    # training loop
    for idx, (inputs, labels1, labels2, labels3) in tqdm(enumerate(train_loader), total=len(train_loader)):
        
        # to GPU
        inputs, labels1, labels2, labels3 = inputs.to(device), labels1.to(device), labels2.to(device), labels3.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward
        outputs1, outputs2, outputs3 = model(inputs.unsqueeze(1).float())
        loss1 = 0.7 * criterion(outputs1, labels1)
        loss2 = 0.1 * criterion(outputs2, labels2)
        loss3 = 0.2 * criterion(outputs3, labels3)
        running_loss += loss1.item() + loss2.item() + loss3.item()
        running_acc += (outputs1.argmax(1)==labels1).float().mean()
        running_acc += (outputs2.argmax(1)==labels2).float().mean()
        running_acc += (outputs3.argmax(1)==labels3).float().mean()
        running_recall += macro_recall_multi(outputs1, labels1, outputs2, labels2, outputs3, labels3)
        
        # backward
        (loss1 + loss2 + loss3).backward()
        
        # optimize
        optimizer.step()
        scheduler.step()
        
    train_loss = running_loss / len(train_loader)
    train_acc = running_acc / (len(train_loader) * 3)
    train_recall = running_recall / len(train_loader)
    
    print('train epoch  : {}'.format(epoch))
    print('      loss   : {:.4f}'.format(train_loss))
    print('      acc    : {:.4f}'.format(train_acc))
    print('      recall : {:.4f}'.format(train_recall))

    history.loc[epoch, 'train_loss'] = train_loss
    history.loc[epoch, 'train_acc'] = train_acc.cpu().numpy()
    history.loc[epoch, 'train_recall'] = train_recall
    
    torch.save(model.state_dict(), out_dir/f'efficientnet-b0_{epoch}.pth')
    
    # --- validation start ---
    if train_size < 1.0:
        torch.cuda.empty_cache()
        gc.collect()
        
        running_loss, running_acc, running_recall = 0.0, 0.0, 0.0
        valid_loss, valid_acc, valid_recall = 0.0, 0.0, 0.0
        model.eval()
        
        with torch.no_grad():
            
            for idx, (inputs, labels1, labels2, labels3) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
                
                # to GPU
                inputs, labels1, labels2, labels3 = inputs.to(device), labels1.to(device), labels2.to(device), labels3.to(device)
                
                # forward
                outputs1, outputs2, outputs3 = model(inputs.unsqueeze(1).float())
                loss1 = 0.50 * criterion(outputs1, labels1)
                loss2 = 0.25 * criterion(outputs2, labels2)
                loss3 = 0.25 * criterion(outputs3, labels3)
                running_loss += loss1.item() + loss2.item() + loss3.item()
                running_acc += (outputs1.argmax(1)==labels1).float().mean()
                running_acc += (outputs2.argmax(1)==labels2).float().mean()
                running_acc += (outputs3.argmax(1)==labels3).float().mean()
                running_recall += macro_recall_multi(outputs1, labels1, outputs2, labels2, outputs3, labels3)
                
        valid_loss = running_loss / len(valid_loader)
        valid_acc = running_acc / (len(valid_loader) * 3)
        valid_recall = running_recall / len(valid_loader)
        
        print('valid epoch  : {}'.format(epoch))
        print('      loss   : {:.4f}'.format(valid_loss))
        print('      acc    : {:.4f}'.format(valid_acc))
        print('      recall : {:.4f}'.format(valid_recall))
        
        history.loc[epoch, 'valid_loss'] = valid_loss
        history.loc[epoch, 'valid_acc'] = valid_acc.cpu().numpy()
        history.loc[epoch, 'valid_recall'] = valid_recall
        
        if valid_recall > best_valid_recall:
            print(f'validation recall has increased from: {best_valid_recall:.4f} to: {valid_recall:.4f}. Saving checkpoint')
            torch.save(model.state_dict(), out_dir/f'{model_name}_{epoch}.pth')
            best_valid_recall = valid_recall

In [33]:
train_history = log_report.get_dataframe()
train_history.to_csv(outdir / 'log.csv', index=False)

train_history

,epoch,iteration,train/loss,train/loss_grapheme,train/loss_vowel,train/loss_consonant,train/acc_grapheme,train/acc_vowel,train/acc_consonant,train/recall,valid/loss,valid/loss_grapheme,valid/loss_vowel,valid/loss_consonant,valid/acc_grapheme,valid/acc_vowel,valid/acc_consonant,valid/recall,lr,elapsed_time
0,1,1883,3.042697,1.797872,0.632122,0.612704,0.601449,0.824641,0.851282,0.677516,0.848359,0.480272,0.173948,0.194139,0.875774,0.960387,0.960526,0.909214,0.001,2243.661856
1,2,3766,1.158309,0.645604,0.287593,0.225112,0.837768,0.912725,0.938502,0.873624,0.695029,0.378965,0.182676,0.133387,0.896022,0.942252,0.966488,0.916853,0.001,4485.461718
2,3,5649,0.907438,0.502178,0.232452,0.172807,0.871242,0.927178,0.948852,0.898889,0.494653,0.275289,0.122158,0.097206,0.927242,0.966032,0.973413,0.939164,0.001,6727.550425
3,4,7532,0.787359,0.430356,0.206906,0.150097,0.887900,0.933570,0.953656,0.912072,0.408498,0.241828,0.092366,0.074304,0.935774,0.976637,0.979911,0.946010,0.001,8968.730490
4,5,9415,0.699629,0.377690,0.187407,0.134532,0.900114,0.939088,0.958386,0.920294,0.412999,0.236399,0.102951,0.073649,0.938056,0.972619,0.980060,0.951533,0.001,11209.000478
5,6,11298,0.641063,0.340617,0.176144,0.124302,0.909256,0.941613,0.960942,0.927734,0.409563,0.246129,0.092293,0.071141,0.934236,0.976091,0.979673,0.946735,0.001,13449.830865
6,7,13181,0.588314,0.309447,0.164080,0.114787,0.916313,0.945563,0.964307,0.933129,0.350197,0.194251,0.096681,0.059265,0.947093,0.972679,0.984028,0.950324,0.001,15688.977573
7,8,15064,0.552341,0.288925,0.155283,0.108134,0.921472,0.948023,0.966103,0.937743,0.332787,0.198453,0.075235,0.059099,0.946687,0.979911,0.982748,0.955584,0.001,17927.412009
8,9,16947,0.523410,0.270066,0.149229,0.104115,0.925769,0.949912,0.967451,0.940794,0.338975,0.199955,0.080003,0.059017,0.945744,0.978571,0.982540,0.958276,0.001,20167.192780
9,10,18830,0.491826,0.250700,0.143343,0.097783,0.930137,0.951868,0.968854,0.944074,0.317771,0.193612,0.070374,0.053785,0.948274,0.981300,0.985972,0.957762,0.001,22406.465798


In [97]:
exit()